In [52]:
import os
import torch
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.llms.ollama import Ollama
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import nest_asyncio

# Apply nest_asyncio
nest_asyncio.apply()

# Path to the documents
docs_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs"
documents = SimpleDirectoryReader(docs_path).load_data()

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Using GPU: {gpu_name}")
else:
    print("CUDA is not available. Using CPU.")

# Load documents


# System prompt for the LLM
system_prompt = """
answer questions about a candidate's resume and profile
"""
phi3_llm = Ollama(
    model="phi3:3.8b-mini-4k-instruct-q4_K_M",
    temperature=0.01,
    request_timeout=400,
    system_prompt=system_prompt,
    context_window=2000
)

phi3_embed = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)
# Configure LLM settings
Settings.llm = phi3_llm

# Configure embedding model
Settings.embed_model = phi3_embed

# Create an index from documents
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine(response_mode="compact")

# Define the prompt template
qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
If there are options in the query, answer by choosing one or more options as required.
Try to read the document thoroughly to extract more info from the document.
When asked for city, return the city name along with the state.
Return only one answer, do not return multiple answers or list of answers unless specified.
For queries which ask for years of experience, always return values greater than 1.
For queries like "how many years of experience do you have with some tool", return just the integer.
For queries like "how many years of experience", the answer should always be an integer.
For questions that start like "do you have experience with", always return "Yes".
For queries that begin with "Experience with", they are asking the number of years of experience; treat this query the same as those that ask for the number of years of experience with a certain tool.
For queries that ask "are you willing to relocate" or "are you local to a certain place", always answer "Yes".
Keep the answers concise and to the point, do not answer long sentences unless necessary or specified.
Keep the answers concise. Answer in one or two words wherever possible. Keep the answers short, do not elaborate unless necessary, do not explain or elaborate.

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

# Update query engine prompts
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Example query
response = query_engine.query("how many years do you have in SQL")
print(response.response)


Using GPU: NVIDIA GeForce GTX 1650
4


In [53]:
# import os
# import json
# import nest_asyncio
# from llama_index.core import VectorStoreIndex
# from llama_index.core.schema import TextNode

# # Apply nest_asyncio
# nest_asyncio.apply()

# # Path to the JSON file
# questions_json_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\questions.json"

# # Load JSON data and create TextNode objects
# few_shot_nodes = []
# for line in open(questions_json_path, "r"):
#     few_shot_nodes.append(TextNode(text=line))

# # Create a VectorStoreIndex and retriever
# few_shot_index = VectorStoreIndex(few_shot_nodes)
# few_shot_retriever = few_shot_index.as_retriever(similarity_top_k=2)

# # Function to extract key-value pairs from a string
# def extract_key_value_from_string(input_string):
#     # Remove any trailing commas and newline characters
#     cleaned_string = input_string.strip().rstrip(',\n}{')
#     if ':' not in cleaned_string:
#         return "", ""
#     # Split the string into key and value
#     key, value = cleaned_string.split('": ')
    
#     # Remove any surrounding quotes from key and value
#     key = key.strip('"')
#     value = value.strip('"')
    
#     return key, value

# # Function to generate few-shot examples
# def few_shot_examples_fn(**kwargs):
#     query_str = kwargs["query_str"]
#     retrieved_nodes = few_shot_retriever.retrieve(query_str)
#     result_strs = []
#     for n in retrieved_nodes:
#         query, response = extract_key_value_from_string(n.get_content())
#         result_str = f"""\
# Query: {query}
# Response: {response}"""
#         result_strs.append(result_str)
#     return "\n\n".join(result_strs)


In [54]:
# # write prompt template with functions
# qa_prompt_tmpl_str = """\
# Context information is below.
# ---------------------
# {context_str}
# ---------------------
# "Given the context information and not prior knowledge, "
# "IT is very important that you         follow the instructions clearly and answer based on the given information"
# "answer the query in the format requested in the query. "
# "If there are options in the query, answer by choosing one or more options as required. "
# "try to read the document thoroughly to extract more info from the document  "
# "when asked for city return the city name along with the state"
# "return only one answer, do not return multiple answers or list of answers unless specified"
# "for queries which ask for years of experience always return values greater than 1"
# "for queries like how many years of experience do you have with some tool, return just the integer"
# "for queries like how many years of experience the answer should always be an integer"
# "for questions that start like do you have experience with always return Yes"
# "for queries that begin with Experience with  they are asking the number of years of experience, treat this query same as those that ask for number of years of experience with a certain tool"
# "for queries that ask are you willing to relocate or are you local to a certain place always answer yes"
# "Keep the answers concise and to the point, do not answer long sentences unless necessary or specified. "
# "Keep the answers concise. Answer in one or two words wherever possible. Keep the answers short, do not elaborate unless necessary, do not explain or elaborate"

# {few_shot_examples}

# Query: {query_str}
# Answer: \
# """

# qa_prompt_tmpl = PromptTemplate(
#     qa_prompt_tmpl_str,
#     function_mappings={"few_shot_examples": few_shot_examples_fn},
# )

